In [1]:
import numpy as np
import pandas as pd

In [2]:
table = np.array([
        [0,1,0,0,1],
        [1,0,1,0,0],
        [0,0,1,1,1],
        [0,0,1,0,0],
        [0,0,0,1,0],
        [1,0,0,1,1],
        [1,0,0,0,1],
        [0,1,1,1,0],
        [0,1,1,0,0],
        [1,1,1,0,1],
        [1,1,0,1,1],
        [0,0,0,0,1]
        ])
"""table = np.array([
        [0,0,1,1,1,0],
        [0,1,0,1,1,1],
        [0,1,1,0,1,0],
        [0,1,1,1,1,1],
        [1,0,0,1,1,0],
        [1,1,0,1,1,1],
        [1,1,1,0,1,0],
        [1,1,1,1,1,1],
        [1,0,1,1,0,0],
        [1,1,0,1,0,0],
        ])"""
df = pd.DataFrame(table, columns=[f'f{i}' for i in range(len(table[0])-1)]+['y'])
df

,f0,f1,f2,f3,y
0,0,1,0,0,1
1,1,0,1,0,0
2,0,0,1,1,1
3,0,0,1,0,0
4,0,0,0,1,0
5,1,0,0,1,1
6,1,0,0,0,1
7,0,1,1,1,0
8,0,1,1,0,0
9,1,1,1,0,1


In [3]:
def listToString(cols):  
    temp = ""
    if isinstance(cols[0],tuple):
        for col in cols:
            temp += f" and f{col[0]} == {col[1]}"
    else:
        temp += f" and f{cols[0]} == {cols[1]}"
    return temp + ')'

def bestFirst(df,colums):
    return max([(df.query(f'f{str(i)} == {j} and y==1').shape[0]/df.query(f'f{str(i)} == {j} and y == 0').shape[0],i,j) for i,j in colums])[1:]

def bestScondary(df,colums,fcol):
    heuristics = [(df.query(f'f{str(i)} == {j} and y == 1').shape[0],i,j) for i,j in colums]
    best_heuristic = max(heuristics)
    best_heuristics =  [heuristic[1:] for heuristic in heuristics if heuristic[0] == best_heuristic[0]]
    return min([(df.query(f" not( f{fcol[0]} == {fcol[1]} and f{i} == {j} ) and y==1").shape[0],i,j) for i,j in best_heuristics])[1:]

In [4]:
def output(*arg):
    for r in h:
        approved = 0
        for el in r:
            if isinstance(el[0],tuple):
                for second in el:
                    f,val = second
                    if arg[f] == val:
                        approved +=1
            else:
                f,val = el
                if arg[f] == val:
                        approved +=1
        if approved == len(r) + (len(r[1])-1 if isinstance(r[1][0],tuple) else 0):
            return 1
    return 0

In [ ]:
h, iterations=[],[]
colums = [(i,j) for j in range(2) for i in range(df.shape[1]-1)]#ako j e 0 togas e complement

while(not df.query('y==1').empty):
    fcol = bestFirst(df,colums)
    zeros = df.query(f"f{fcol[0]} == {fcol[1]} and y == 0") 
    scols=[]
    for row in zeros.itertuples():
        posible = [(j,not col) for j,col in enumerate(row[1:-1]) if j != fcol[0]]
        scols.append(bestScondary(df,posible,fcol))
    h.append((fcol, tuple(scols)))    
    df = df.query(f" not( f{fcol[0]} == {fcol[1]} " + listToString(scols))
    iterations.append(df)

In [16]:
output(0,0,1,1)

1

In [17]:
for iteration in iterations:
    print(iteration.query("y==1"))

    f0  f1  f2  f3  y
0    0   1   0   0  1
2    0   0   1   1  1
9    1   1   1   0  1
11   0   0   0   0  1
   f0  f1  f2  f3  y
2   0   0   1   1  1
9   1   1   1   0  1
   f0  f1  f2  f3  y
2   0   0   1   1  1
Empty DataFrame
Columns: [f0, f1, f2, f3, y]
Index: []


In [18]:
h

[((2, 0), ((0, True),)),
 ((2, 0), ((3, False),)),
 ((0, 1), ((1, True),)),
 ((3, 1), ((2, True), (1, False)))]